In [ ]:
#Inital all lines and code are similar to solution 1, the differnece is only in the calucation.
import cv2                                                                                      
import mediapipe as mp                                                                          
import numpy as np

# 2. Initialize MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose    = mp.solutions.pose

# 3. Open webcam feed
cap = cv2.VideoCapture(0)                                                                      

# 4. Variables to track jump
ground_y   = None      # baseline (standing) nose y-coordinate in pixels
peak_y     = None      # minimum y (highest point) during jump
jumping    = False

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip and convert to RGB for MediaPipe
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # 5. Extract nose landmark and convert to pixel coords
            h, w, _ = image.shape
            nose_landmark = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
            nose_y = int(nose_landmark.y * h)

            # 6. Establish ground baseline when not jumping
            if not jumping:
                if ground_y is None:
                    ground_y = nose_y
                else:
                    # update baseline to the highest consistent point
                    ground_y = (ground_y * 0.9) + (nose_y * 0.1)

            # 7. Detect start of jump: nose moves upward beyond a threshold
            if not jumping and ground_y - nose_y > 30:
                jumping = True
                peak_y = nose_y  # initialize peak

            # 8. While jumping, track the highest point (smallest y value)
            if jumping:
                peak_y = min(peak_y, nose_y)
                # Detect landing: nose returns close to baseline
                if nose_y - ground_y < 30:
                    # compute jump height in pixels
                    jump_height_px = ground_y - peak_y
                    cv2.putText(image,
                                f"Jump height: {jump_height_px}px",
                                (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                1, (0, 255, 0), 2, cv2.LINE_AA)
                    # reset for next jump
                    jumping = False
                    peak_y = None

            # Visualize current nose y
            cv2.circle(image, (int(nose_landmark.x*w), nose_y), 8, (0,0,255), -1)

        # 9. Display
        cv2.imshow('Jump Height Estimation', image)
        if cv2.waitKey(5) & 0xFF == 27:  # press ESC to exit
            break

cap.release()
cv2.destroyAllWindows()
